# Demo story

out of all the possible events that can give rise to an infinite number of chains, we are just simulating and demoing just one chain : delay, crew legality, substitution, repositioning, hotel/transport, policy fallback

delay → legality check → substitution → repositioning → hotel/transport → policy fallback

- Event 1: "UA123 delayed 90 min"
- Event 2: "UA123 delay increases to 210 min"
- Event 3: "SFO spare crew found"
- Event 4: "SFO fog — reposition flight canceled"
- Event 5: "DEN crew identified"
- Event 6: "No reposition flight from DEN"
- Event 7: "Hotel at ORD overbooked"
- Event 8: "Alternative hotel secured + notifications sent"

1. The core of the simulation is **time-driven flight status data**.
    
    You will create a mock flight schedule where:
    
- Flights have planned departure and arrival times.
- As the simulation clock advances, some flights develop delays (you modify delay_minutes dynamically as part of the data update).
- These updates happen independently of any direct agent involvement — the agent only sees the changed data when it queries.
1. The agent’s job is to **continuously monitor the flight data**.
    
    At each clock tick:
    
- The agent queries flight status for flights under its watch.
- The agent notices delay or disruption based on the current data state, not because an external event handed it that knowledge.
1. Other data (crew roster, hotel, repositioning options) is mostly static but still queried as needed.
- Crew data defines duty limits, assigned flights, qualifications.
- Hotel and reposition flight data could also change (for realism) but are not monitored continuously; they are checked when the agent’s reasoning requires.
1. The clock powers the data simulation:
- The clock advances simulated time (could be 1 minute per loop or whatever scale works for demo).
- A separate data update function updates flight delay fields as time advances. For example, after 10 ticks, flight UA123’s delay_minutes field is updated to 90. After 20 ticks, it’s 210.
- The agent doesn’t know these thresholds. It just keeps querying flight data and decides when action is needed.
1. There’s no artificial event queue like “fog at SFO” for the agent to process.
- All the agent sees is that repositioning flight from SFO is delayed or unavailable when it queries.
- The cause (fog) is irrelevant to the agent’s reasoning in this case.

# Agent Buckets

1️⃣ **Status Query Agents**

- Scope: Gather and summarize current state.
- Examples: Flight status agent, crew status agent, weather condition agent.
- Purpose: Provide situational awareness, inform other agents' decisions.
- Nature: Passive/reactive, no direct action beyond informing.

2️⃣ **Action Executor Agents**

- Scope: Carry out specific operational tasks or commands.
- Examples: Crew assigner, hotel booker, transport arranger, communicator.
- Purpose: Translate decisions into changes in the world or system state.
- Nature: Active/doers, often invoked by reasoning agents.

3️⃣ **Rule Evaluator Agents**

- Scope: Check compliance with legal, contractual, or operational rules.
- Examples: Duty legality checker, qualification checker, policy retriever.
- Purpose: Gatekeeper for valid actions, ensure plans follow rules.
- Nature: Deciders/validators.

The current categories form the *foundation*. Other domains reflect either meta-control (planner, optimizer) or domain extensions (customer comms, negotiator) as your system expands.

---

### For demo crew rescheduling:

✅ These three agent domains — **status queries, action executors, rule evaluators** — cover the functional scope needed for the core of the problem:

- You need to observe the situation.
- You need to validate legal and operational constraints.
- You need to act (assign, move, rest, inform).

In this domain:

- Most of the work boils down to these roles.
- Additional agents would usually *fit inside* these categories or serve as **orchestrators/planners** that coordinate among them.

---

Other domains are absolutely possible



### Mock data template

In [37]:
import pandas as pd

#  Flight Schedule 
flight_schedule_df = pd.DataFrame([
    {
        "flight_id": "UA123",
        "origin": "ORD",
        "destination": "SFO",
        "sched_dep": "2024-08-10 08:00",
        "sched_arr": "2024-08-10 11:00",
        "aircraft_type": "B737",
        "delay_minutes": 210,  # 3.5 hr delay
        "status": "delayed",
        "gate": "C5",
        "remarks": "ground stop"
    },
    {
        "flight_id": "UA456",
        "origin": "SFO",
        "destination": "DEN",
        "sched_dep": "2024-08-10 12:00",
        "sched_arr": "2024-08-10 14:00",
        "aircraft_type": "B737",
        "delay_minutes": 0,
        "status": "ontime",
        "gate": "B12",
        "remarks": ""
    }
])

#  Crew Roster 
crew_roster_df = pd.DataFrame([
    {
        "crew_id": "C001",
        "name": "Jane Doe",
        "role": "captain",
        "base": "ORD",
        "qualified_aircraft": "B737",
        "assigned_flight_id": "UA123",
        "duty_start": "2024-08-10 07:00",
        "duty_end": "2024-08-10 15:00",
        "rest_until": "2024-08-10 23:00",
        "status": "active"
    },
    {
        "crew_id": "C002",
        "name": "John Roe",
        "role": "FO",
        "base": "ORD",
        "qualified_aircraft": "B737",
        "assigned_flight_id": "UA123",
        "duty_start": "2024-08-10 07:00",
        "duty_end": "2024-08-10 15:00",
        "rest_until": "2024-08-10 23:00",
        "status": "active"
    },
    {
        "crew_id": "C010",
        "name": "Sam Lee",
        "role": "FO",
        "base": "SFO",
        "qualified_aircraft": "B737",
        "assigned_flight_id": None,
        "duty_start": None,
        "duty_end": None,
        "rest_until": "2024-08-10 06:00",
        "status": "active"
    },
    {
        "crew_id": "C011",
        "name": "Alex Kim",
        "role": "FO",
        "base": "DEN",
        "qualified_aircraft": "B737",
        "assigned_flight_id": None,
        "duty_start": None,
        "duty_end": None,
        "rest_until": "2024-08-10 04:00",
        "status": "active"
    }
])

#  Hotel Inventory 
hotel_inventory_df = pd.DataFrame([
    {
        "hotel_id": "H001",
        "name": "ORD Airport Hotel",
        "location": "ORD",
        "capacity": 200,
        "available_rooms": 0
    },
    {
        "hotel_id": "H002",
        "name": "ORD Downtown Inn",
        "location": "ORD",
        "capacity": 150,
        "available_rooms": 10
    },
    {
        "hotel_id": "H003",
        "name": "SFO Airport Hotel",
        "location": "SFO",
        "capacity": 100,
        "available_rooms": 5
    },
    {
        "hotel_id": "H004",
        "name": "DEN Airport Hotel",
        "location": "DEN",
        "capacity": 120,
        "available_rooms": 8
    }
])

#  Repositioning Flights 
repositioning_flights_df = pd.DataFrame([
    {
        "flight_id": "UA9001",
        "origin": "DEN",
        "destination": "SFO",
        "sched_dep": "2024-08-10 10:00",
        "sched_arr": "2024-08-10 12:00",
        "seats_available": True
    },
    {
        "flight_id": "UA9002",
        "origin": "DEN",
        "destination": "ORD",
        "sched_dep": "2024-08-10 11:00",
        "sched_arr": "2024-08-10 13:00",
        "seats_available": False
    },
    {
        "flight_id": "UA9003",
        "origin": "SFO",
        "destination": "ORD",
        "sched_dep": "2024-08-10 09:00",
        "sched_arr": "2024-08-10 15:00",
        "seats_available": True
    }
])

crew_duty_logs_df = pd.DataFrame(
[{'crew_id': 'C001',
  'duty_end': '2024-08-01 17:00:00',
  'duty_start': '2024-08-01 08:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-02 18:00:00',
  'duty_start': '2024-08-02 06:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-03 18:00:00',
  'duty_start': '2024-08-03 08:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-04 18:00:00',
  'duty_start': '2024-08-04 06:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-05 15:00:00',
  'duty_start': '2024-08-05 07:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-06 16:00:00',
  'duty_start': '2024-08-06 08:00:00'},
 {'crew_id': 'C001',
  'duty_end': '2024-08-07 18:00:00',
  'duty_start': '2024-08-07 08:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-01 17:00:00',
  'duty_start': '2024-08-01 09:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-02 17:00:00',
  'duty_start': '2024-08-02 07:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-03 16:00:00',
  'duty_start': '2024-08-03 07:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-04 18:00:00',
  'duty_start': '2024-08-04 07:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-05 16:00:00',
  'duty_start': '2024-08-05 07:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-06 12:00:00',
  'duty_start': '2024-08-06 06:00:00'},
 {'crew_id': 'C002',
  'duty_end': '2024-08-07 16:00:00',
  'duty_start': '2024-08-07 09:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-01 17:00:00',
  'duty_start': '2024-08-01 07:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-02 15:00:00',
  'duty_start': '2024-08-02 09:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-03 14:00:00',
  'duty_start': '2024-08-03 06:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-04 20:00:00',
  'duty_start': '2024-08-04 08:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-05 16:00:00',
  'duty_start': '2024-08-05 07:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-06 21:00:00',
  'duty_start': '2024-08-06 09:00:00'},
 {'crew_id': 'C010',
  'duty_end': '2024-08-07 18:00:00',
  'duty_start': '2024-08-07 07:00:00'}]
)

transport_df = pd.DataFrame([
    {
        "from_location": "Downtown Marriott",
        "to_location": "SFO",
        "mode": "cab",
        "eta_minutes": 45,
        "available_vehicles": 3
    },
    {
        "from_location": "Union Square Hotel",
        "to_location": "ORD",
        "mode": "shuttle",
        "eta_minutes": 30,
        "available_vehicles": 2
    },
    {
        "from_location": "Central Train Station",
        "to_location": "DEN",
        "mode": "train",
        "eta_minutes": 120,
        "available_vehicles": 5
    },
    {
        "from_location": "Airport Terminal 3",
        "to_location": "SFO",
        "mode": "shuttle",
        "eta_minutes": 20,
        "available_vehicles": 4
    },
    {
        "from_location": "Convention Center",
        "to_location": "ORD",
        "mode": "cab",
        "eta_minutes": 60,
        "available_vehicles": 1
    }
])


In [38]:
transport_df

,from_location,to_location,mode,eta_minutes,available_vehicles
0,Downtown Marriott,SFO,cab,45,3
1,Union Square Hotel,ORD,shuttle,30,2
2,Central Train Station,DEN,train,120,5
3,Airport Terminal 3,SFO,shuttle,20,4
4,Convention Center,ORD,cab,60,1


### Functions that Agent will use

In [43]:
def query_flight_status(flight_id, flight_schedule_df):

    """
    Return status information for the given flight ID.

    Args:

    flight_id (str): Flight identifier.

    flight_schedule_df (DataFrame): Flight schedule data source.

    Returns:

    dict: { flight_id, status, delay_minutes, gate, remarks, aircraft_type }

    """

    if flight_id is None:
        raise ValueError("Please provide flight_schedule_df")
        
    cols = ['flight_id', 'status', 'delay_minutes', 'gate', 'remarks', 'aircraft_type']
    filtered = flight_schedule_df[flight_schedule_df['flight_id'] == flight_id][cols]
    result = filtered.to_dict(orient = 'records')

    return result[0] if result else None

    pass



def query_crew_roster(flight_id=None, crew_id=None, crew_roster_df=None):

    """

    Fetch assigned crew for a flight or details for a specific crew member.

    Args:

    flight_id (str, optional): Flight identifier.

    crew_id (str, optional): Crew identifier.

    crew_roster_df (DataFrame): Crew data source.

    Returns:

    list of dicts or dict: Crew details.

    """
    if crew_roster_df is None:
        raise ValueError("Please provide `crew_roster_df`")

    df = crew_roster_df.copy()

    if flight_id is not None and crew_id is not None:
        raise ValueError("Please supply one of flight_id or crew_id")
    elif flight_id is not None:
        return df[df['assigned_flight_id'] == flight_id].to_dict(orient='records')
    elif crew_id is not None:
        result = df[df['crew_id'] == crew_id].to_dict(orient = 'records')
        return result[0] if result else None
    else:
        raise ValueError("Please supply flight id or crew id")

    return result

    pass



def query_spare_pool(location, aircraft_type, role, crew_roster_df):

    """

    Find available spare crew at location with given qualification.

    Args:

    location (str)

    aircraft_type (str)

    role (str)

    crew_roster_df (DataFrame)

    Returns:

    list of dicts: Spare crew candidates.

    """

    if crew_roster_df is None:
        raise ValueError("Please provide `crew_roster_df`")

    df = crew_roster_df.copy()

    if location is None or aircraft_type is None or role is None:
        raise ValueError("Please supply location, aircraft type, and role")
    else:
        conditions = (
            (df['base'] == location) & 
            (df['qualified_aircraft'] == aircraft_type) & 
            (df['role'] == role) & 
            (df['status'] == 'active') &
            ((df['assigned_flight_id'].isna())| (df['assigned_flight_id'] == "")) &
            (df['rest_until'] <= str(pd.Timestamp.now()))
        )
        result = df.loc[conditions].to_dict(orient='records')
        
    return result
    

    pass



def duty_hour_checker(crew_id, planned_start, planned_end, duty_rules):

    """

    Check legality of crew duty period.

    Args:

    crew_id (str)

    planned_start (datetime)

    planned_end (datetime)

    duty_rules (dict)

    Returns:

    dict: { legal: bool, reason: str if not legal }

    """

    max_duty   = duty_rules["max_duty_hours"]
    min_rest   = duty_rules["min_rest_hours"]
    flight_ac  = duty_rules["aircraft_type"]
    logs_df    = duty_rules["crew_duty_logs_df"]
    roster_df  = duty_rules["crew_roster_df"]

    qual = roster_df.loc[roster_df["crew_id"] == crew_id, "qualified_aircraft"]
    if qual.empty or qual.iloc[0] != flight_ac:
        return {"legal": False, "reason": f"Not qualified for {flight_ac}"}

    logs_df = logs_df.copy()
    logs_df["duty_start"] = pd.to_datetime(logs_df["duty_start"])
    logs_df["duty_end"]   = pd.to_datetime(logs_df["duty_end"])

    past = logs_df[
        (logs_df["crew_id"] == crew_id) &
        (logs_df["duty_end"] <= planned_start)
    ]
    if not past.empty:
        last_end = past["duty_end"].max()
        rest_hrs = (planned_start - last_end).total_seconds() / 3600.0
        if rest_hrs < min_rest:
            return {
                "legal": False,
                "reason": f"Only {rest_hrs:.1f}h rest (<{min_rest}h required)"
            }

    # 2) Max‐duty‐hours check
    duty_hrs = (planned_end - planned_start).total_seconds() / 3600.0
    if duty_hrs > max_duty:
        return {
            "legal": False,
            "reason": f"{duty_hrs:.1f}h duty (> {max_duty}h limit)"
        }

    # passed all checks
    return {"legal": True}


    pass



def reposition_flight_finder(from_location, to_location, repositioning_flights_df):

    """

    Find repositioning flight options for crew.

    Args:

    from_location (str)

    to_location (str)

    repositioning_flights_df (DataFrame)

    Returns:

    list of dicts: Flight options.

    """

    if repositioning_flights_df is None:
        raise ValueError("Please provide `repositioning_flights_df`")
    if not (from_location and to_location):
        raise ValueError("Please supply both from_location and to_location")

    df = repositioning_flights_df.copy()

    for col in ("sched_dep", "sched_arr"):
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])

    conditions = (
        (df["origin"] == from_location) &
        (df["destination"] == to_location) &
        (df["seats_available"] == True)
    )
    

    filtered = df.loc[conditions].sort_values(by="sched_dep")

    # Convert to list of dicts
    return filtered.to_dict(orient="records")

    pass



def book_hotel(location, crew_id, hotel_inventory_df):

    """

    Attempt to book hotel for crew.

    Args:

    location (str)

    crew_id (str)

    hotel_inventory_df (DataFrame)

    Returns:

    dict: { success: bool, hotel_id or failure_reason }

    """

    if hotel_inventory_df is None:
        raise ValueError("Please provide `hotel_inventory_df`")

    # Find all hotels at the location with rooms available
    mask = (
        (hotel_inventory_df["location"] == location) &
        (hotel_inventory_df["available_rooms"] > 0)
    )
    candidates = hotel_inventory_df.loc[mask]

    if candidates.empty:
        return {
            "success": False,
            "failure_reason": f"No rooms available at {location}"
        }

    # Choose the hotel with the most available rooms
    chosen_idx = candidates["available_rooms"].idxmax()
    chosen_hotel = hotel_inventory_df.loc[chosen_idx, "hotel_id"]

    # Decrement inventory
    hotel_inventory_df.at[chosen_idx, "available_rooms"] -= 1

    return {
        "success": True,
        "hotel_id": chosen_hotel
    }

    pass



def arrange_transport(from_location, to_location, crew_id):

    """

    Arrange transport for crew.

    Args:

    from_location (str)

    to_location (str)

    crew_id (str)

    Returns:

    dict: { success: bool, eta or failure_reason }

    """

    # Filter matching routes with available vehicles
    available = transport_df[
        (transport_df["from_location"]  == from_location)   &
        (transport_df["to_location"]  == to_location)   &
        (transport_df["available_vehicles"] > 0)
    ]
    if available.empty:
        return {
            "success": False,
            "failure_reason": f"No transport available from {from_location} to {to_location}"
        }
    
    # Choose the fastest option
    best_idx = available["eta_minutes"].idxmin()
    best_option = transport_df.loc[best_idx]
    
    # Decrement availability
    transport_df.at[best_idx, "available_vehicles"] -= 1
    
    return {
        "success": True,
        "eta_minutes": best_option["eta_minutes"],
        "mode": best_option["mode"]
    }

    pass



def policy_retriever(situation_description, policy_data):

    """

    Fetch policy guidance for situation.

    Args:

    situation_description (str)

    policy_data (dict or list)

    Returns:

    dict: { policy_text, recommended_action }

    """

    pass



def check_aircraft_assignment_change(flight_id, flight_schedule_df):

    """

    Detect if aircraft type has changed for flight.

    Args:

    flight_id (str)

    flight_schedule_df (DataFrame)

    Returns:

    dict: { aircraft_type, changed: bool }

    """

    pass



def check_weather_conditions(airport_code, weather_data):

    """

    Get weather summary and delay risk.

    Args:

    airport_code (str)

    weather_data (dict or DataFrame)

    Returns:

    dict: { summary, risk_level }

    """

    pass



def check_crew_future_assignment(crew_id, crew_roster_df):

    """

    Check future assignment conflicts for spare crew.

    Args:

    crew_id (str)

    crew_roster_df (DataFrame)

    Returns:

    dict: { next_assignment, conflict: bool }

    """

    pass



def send_notification(message_text, priority):

    """

    Send structured notification to ops team.

    Args:

    message_text (str)

    priority (str)

    Returns:

    dict: { sent: bool, timestamp }

    """

    pass

In [25]:
# query flight status
flight_id = "UA123"
query_flight_status(flight_id, flight_schedule_df)

{'flight_id': 'UA123',
 'status': 'delayed',
 'delay_minutes': 210,
 'gate': 'C5',
 'remarks': 'ground stop',
 'aircraft_type': 'B737'}

In [26]:
# query related crews
query_crew_roster(flight_id = flight_id, crew_roster_df = crew_roster_df)

[{'crew_id': 'C001',
  'name': 'Jane Doe',
  'role': 'captain',
  'base': 'ORD',
  'qualified_aircraft': 'B737',
  'assigned_flight_id': 'UA123',
  'duty_start': '2024-08-10 07:00',
  'duty_end': '2024-08-10 15:00',
  'rest_until': '2024-08-10 23:00',
  'status': 'active'},
 {'crew_id': 'C002',
  'name': 'John Roe',
  'role': 'FO',
  'base': 'ORD',
  'qualified_aircraft': 'B737',
  'assigned_flight_id': 'UA123',
  'duty_start': '2024-08-10 07:00',
  'duty_end': '2024-08-10 15:00',
  'rest_until': '2024-08-10 23:00',
  'status': 'active'}]

In [27]:
# query available crews
query_spare_pool(location = "SFO", aircraft_type = 'B737', role = 'FO', crew_roster_df = crew_roster_df)

[{'crew_id': 'C010',
  'name': 'Sam Lee',
  'role': 'FO',
  'base': 'SFO',
  'qualified_aircraft': 'B737',
  'assigned_flight_id': None,
  'duty_start': None,
  'duty_end': None,
  'rest_until': '2024-08-10 06:00',
  'status': 'active'}]

In [28]:
# query legality of crew duty
from datetime import datetime, timedelta
duty_rules = {
    "max_duty_hours":     8.0,
    "min_rest_hours":    10.0,
    "aircraft_type":    "B737",
    "crew_duty_logs_df":  crew_duty_logs_df,
    "crew_roster_df":     crew_roster_df
}

# 4) Test a planned duty
start = datetime(2024, 8, 3, 8, 0)
end   = start + timedelta(hours=7)
print(duty_hour_checker("C010", start, end, duty_rules))

{'legal': True}


In [45]:
# query available transport
arrange_transport("Downtown Marriott", "SFO", "C010")

{'success': True, 'eta_minutes': 45, 'mode': 'cab'}

In [30]:
# query for available flights for sending available crews
reposition_flight_finder(from_location = "DEN", to_location = "SFO", repositioning_flights_df = repositioning_flights_df)

[{'flight_id': 'UA9001',
  'origin': 'DEN',
  'destination': 'SFO',
  'sched_dep': Timestamp('2024-08-10 10:00:00'),
  'sched_arr': Timestamp('2024-08-10 12:00:00'),
  'seats_available': True}]

In [41]:
# query available hotel for stranded crews
book_hotel(location = "ORD", crew_id = None, hotel_inventory_df = hotel_inventory_df)

{'success': True, 'hotel_id': 'H002'}